In [1]:
from model import PolicyCNNMLP
import time
import torch
import numpy as np
import cv2


In [2]:
# Init Policy
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)
np.random.seed(1337)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda"), "CUDA is not available"
torch.set_float32_matmul_precision('high')

policy = PolicyCNNMLP()
policy = policy.to(device)
use_compile = True
if use_compile:
    policy = torch.compile(policy)
checkpoint_path = '/home/qutrll/data/pot_pick_place_ckpt/1/checkpoint_step_90000_seed_1337.ckpt'
checkpoint = torch.load(checkpoint_path)
policy.load_state_dict(checkpoint['model_state_dict'])


/tmp/ipykernel_63461/3994205791.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


<All keys matched successfully>

In [3]:
# create dummy data
images = torch.rand(1, 3, 3, 240, 424).to(device)
q_pos = torch.rand(1, 7).to(device)
progress = torch.rand(1, 1).to(device)


In [4]:
images.shape, q_pos.shape, progress.shape


(torch.Size([1, 3, 3, 240, 424]), torch.Size([1, 7]), torch.Size([1, 1]))

In [5]:
output = policy(images, q_pos, progress)


In [6]:
output = output.squeeze()
output.shape


torch.Size([8])

In [7]:
joints = output[0:6]
gripper = output[6]
progress = output[7]


In [11]:
# Convert to numpy
joints = joints.detach().cpu().numpy()
gripper = gripper.detach().cpu().numpy()
progress = progress.detach().cpu().numpy()


In [12]:
joints


array([ 0.35340172, -0.18972424,  0.18311182,  0.53951514,  1.4495711 ,
        0.58404577], dtype=float32)

In [13]:
gripper


array(1.0217807, dtype=float32)

In [14]:
progress


array(0.4581894, dtype=float32)

In [22]:
image_1 = torch.rand(3, 240, 424).to(device)
image_2 = torch.rand(3, 240, 424).to(device)
image_3 = torch.rand(3, 240, 424).to(device)


In [23]:
# combine images
images = torch.stack([image_1, image_2, image_3], dim=0).unsqueeze(0)


In [30]:
action = torch.rand(1,8).to(device)


torch.Size([1])